**Installing the Dependencies**

In [ ]:
from llama_index.llms.groq import Groq
import csv
from openai import OpenAI
from mistralai import Mistral
from transformers import AutoTokenizer

In [ ]:
import json
import re
import pandas as pd

In [ ]:
N=1 #can be 1,2,3,4,5

In [ ]:
with open(f'sample_200/1k_msg_rawdiff_part_{N}.txt', 'r') as file:
    data = file.read()
diffs_label = json.loads(data)

In [ ]:
def is_whitespace_only_diff(diff):
    lines = diff.splitlines()
    added_pattern = re.compile(r'^\+')
    removed_pattern = re.compile(r'^\-')
    for line in lines:
        if added_pattern.match(line) or removed_pattern.match(line):
            # Remove the initial '+' or '-' and compare the content ignoring whitespace
            content = line[1:].strip()
            if content:
                return False
    return True

In [ ]:
labels = list(diffs_label.keys())
diffs = list(diffs_label.values())

In [ ]:
filtered_diff_label = []

Filter Whitespace Diffs, Bot Commits

In [ ]:
HF_TOKEN = 'Hugging_Face_API_Key'
pattern = re.compile(r'\d+ \. \d+( \. \d+)*')
filtered_labels, filtered_diffs = [],[]
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", token = HF_TOKEN)
for i in range(0,len(labels)):
    n_tokens = len(tokenizer.encode(diffs[i]))
    if (not pattern.search(labels[i])) and (not is_whitespace_only_diff(diffs[i])) and n_tokens<131072:
        filtered_labels.append(labels[i])
        filtered_diffs.append(diffs[i])
        filtered_diff_label.append({'diff':diffs[i], 'label':labels[i]})

In [ ]:
print(len(filtered_diff_label))

In [ ]:
groq_api = 'Groq_API_Key'

Using Groq API for testing with Llama3.1 70B and 8B

In [ ]:
def get_response(model,prompt):
  llm_groq = Groq(model=model, api_key=groq_api)
  sequences = llm_groq.complete(prompt = prompt)
  return sequences.text

Using OpenAI API for testing with GPT models

In [ ]:
OPENAI_API_KEY='Open_AI_API_Key'
client = OpenAI(api_key=OPENAI_API_KEY)
def getGPTResponse(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content

Using Mistral API key for Mistral Large model

In [ ]:
mistral_key = 'Mistral_API_Key'
model = "mistral-large-2402"
client = Mistral(api_key=mistral_key)
def getMistralResponse(prompt):
    chat_response = client.chat.complete(
        model= model,
        messages = [
            {
                "role": "user",
                "content": prompt,
            },
        ]
    )
    return chat_response.choices[0].message.content

Using Full Diff to generate Commit message

In [ ]:
cf_1, cf_2, cf_3, cf_4 = [], [], [], []

Llama3.1-70B

In [ ]:
cf_1=[]
for i in range(0, len(filtered_diff_label)):
  if ('llama-70b-output' not in filtered_diff_label[i] or filtered_diff_label[i]['llama-70b-output'] == 'ERROR'):
    diff=filtered_diff_label[i]['diff']
    prompt = f"The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly. {diff} According to the diff, the commit message should be:"
    try:
      filtered_diff_label[i]['llama-70b-output'] = get_response("llama-3.1-70b-versatile",prompt)
    except:
      cf_1.append(i)
      filtered_diff_label[i]['llama-70b-output'] = 'ERROR'
      pass

Llama3.1-8B

In [ ]:
cf_2=[]
for i in range(0, len(filtered_diff_label)):
  if ('llama3.1-8b-output' not in filtered_diff_label[i] or filtered_diff_label[i]['llama3.1-8b-output'] == 'ERROR'):
    diff=filtered_diff_label[i]['diff']
    prompt = f"The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly. {diff} According to the diff, the commit message should be:"
    try:
      filtered_diff_label[i]['llama3.1-8b-output'] = get_response("llama-3.1-8b-instant",prompt)
    except:
      cf_2.append(i)
      filtered_diff_label[i]['llama3.1-8b-output'] = 'ERROR'
      pass

Mistral-Large

In [ ]:
cf_3=[]
for i in range(0, len(filtered_diff_label)):
    if ('mistral-large-output' not in filtered_diff_label[i] or filtered_diff_label[i]['mistral-large-output'] == 'ERROR'):
        diff=filtered_diff_label[i]['diff']
        prompt = f"{diff} According to the diff, the commit message should be:"
        try:
            filtered_diff_label[i]['mistral-large-output'] = getMistralResponse(prompt)
        except:
            cf_3.append(i)
            filtered_diff_label[i]['mistral-large-output'] = 'ERROR'
            pass

GPT-4o

In [ ]:
cf_4 = []
for i in range(0, len(filtered_diff_label)):
    if('gpt-4o-output' not in filtered_diff_label[i] or filtered_diff_label[i]['gpt-4o-output'] == 'ERROR'):
        diff=filtered_diff_label[i]['diff']
        prompt = f"{diff} According to the diff, the commit message in atmost 20 words should be:"
        try:
            filtered_diff_label[i]['gpt-4o-output'] = getGPTResponse(prompt)
        except:
            cf_4.append(i)
            filtered_diff_label[i]['gpt-4o-output'] = 'ERROR'
            pass

Save Outputs

In [ ]:
csv_file_name = f'Outputs/Total_Diff/diff_output_{N}.csv'

In [ ]:
headers = filtered_diff_label[0].keys()

with open(csv_file_name, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(filtered_diff_label)

Use Split diff for Generating Commit message

Split each diff to multiple files

In [ ]:
new_diff_label=[]
df = pd.read_csv(f'Outputs/Total_Diff/diff_output_{N}.csv')
df_cleaned = df.replace('ERROR', pd.NA).dropna()
filtered_diffs = list(df_cleaned['diff'])
for i in range (0,len(filtered_diffs)):
  result = re.split(r'(?=diff --git)', filtered_diffs[i])
  label = filtered_labels[i]
  for res in result:
      if(len(res)>0):
        new_diff_label.append({'diff':res, 'label':label, 'actual_index':i})

In [ ]:
cf1, cf2, cf3, cf4=[],[],[],[]

Llama3.1-70B

In [ ]:
cf1=[]
for i in range(0, len(new_diff_label)):
  if('llama-70b-output' not in new_diff_label[i] or new_diff_label[i]['llama-70b-output'] == 'ERROR'):
    diff=new_diff_label[i]['diff']
    prompt = f"{diff} According to the diff, the commit message in atmost 20 words should be:"
    try:
      new_diff_label[i]['llama-70b-output'] = get_response("llama-3.1-70b-versatile",prompt)
    except:
      cf1.append(i)
      new_diff_label[i]['llama-70b-output'] = 'ERROR'
      pass

Llama3.1-8B

In [ ]:
for i in range(0, len(new_diff_label)):
  diff=new_diff_label[i]['diff']
  prompt = f"{diff} According to the diff, the commit message should be:"
  try:
    new_diff_label[i]['llama3.1-8b-output'] = get_response("llama-3.1-8b-instant",prompt)
  except:
    cf2.append(i)
    new_diff_label[i]['llama3.1-8b-output'] = 'ERROR'
    pass

Mistral-Large

In [ ]:
flag = True
count = 10
while(flag and count>0):
    cf3=[]
    for i in range (0, len(new_diff_label)):
        if('mistral-large-output' not in new_diff_label[i] or new_diff_label[i]['mistral-large-output'] == 'ERROR'):
            diff=new_diff_label[i]['diff']
            prompt = f"{diff} According to the diff, the commit message in 20 words should be:"
            try:
                new_diff_label[i]['mistral-large-output'] = getMistralResponse(prompt)
            except:
                cf3.append(i)
                new_diff_label[i]['mistral-large-output'] = 'ERROR'
                pass
    csv_file_name = f'Outputs/File_Level_Diff/split_diff_output_{N}.csv'
    headers = new_diff_label[0].keys()
    with open(csv_file_name, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(new_diff_label)
    print(len(cf3))
    if(len(cf3)<=0):
        flag=False
    count-=1


GPT-4o

In [ ]:
cf4=[]
for i in range(0, len(new_diff_label)):
  if('gpt-4o-output' not in new_diff_label[i] or new_diff_label[i]['gpt-4o-output'] == 'ERROR'):
    diff=new_diff_label[i]['diff']
    prompt = f"{diff} According to the diff, the commit message should be:"
    try:
      new_diff_label[i]['gpt-4o-output'] = getGPTResponse(prompt)
    except:
      cf4.append(i)
      new_diff_label[i]['gpt-4o-output'] = 'ERROR'
      pass

In [ ]:
len(cf4)

Save Outputs

In [ ]:
csv_file_name = f'Outputs/File_Level_Diff/split_diff_output_{N}.csv'

In [ ]:
headers = new_diff_label[0].keys()

with open(csv_file_name, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(new_diff_label)

In [ ]:
df = pd.read_csv(csv_file_name)
df_cleaned = df.replace('ERROR', pd.NA).dropna()
df_cleaned.to_csv(csv_file_name, index=False)